In [3]:
from ngsildclient import Entity, Client
import datetime
import numpy as np

In [4]:
host = "10.0.0.1"
client = Client(hostname=host, port=1026, port_temporal=8080)
ctx = "http://nginx:8000/context-ngsi.jsonld"

Connected to Context Broker at 10.0.0.1:1026 | tenant=N/A | vendor=Orion-LD | version=1.1.2

In [8]:
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
cowx = Entity("AnimalDailyObserved", "cow_x", ctx = ctx)
cowx["methaneyield"] = np.random.randint(30, 60)
cowx["net_energy_requirement"] = np.random.randint(700, 1000)
cowx["observedAt"] = start.isoformat() + "Z"
cowx.pprint()
client.create(cowx)

{
  "id": "urn:ngsi-ld:AnimalDailyObserved:cow_x",
  "type": "AnimalDailyObserved",
  "@context": "http://nginx:8000/context-ngsi.jsonld",
  "methaneyield": 45,
  "net_energy_requirement": 991,
  "observedAt": "2023-01-01T00:00:00Z"
}

True

# Build a daily measurement

In [31]:
import json
from dataclasses import dataclass, asdict

@dataclass
class AnimalDay:
    methaneyield: float
    net_energy_requirement: float

@dataclass
class Day:
    observedAt : str
    obs: AnimalDay
    pred: AnimalDay

obs = AnimalDay(np.random.randint(30, 60),
                 np.random.randint(30, 60))
pred = AnimalDay(np.random.randint(30, 60),
                 np.random.randint(30, 60))
obs.__dict__
d = Day(start.isoformat(), obs, pred)
asdict(d)

{'observedAt': '2023-01-01T00:00:00',
 'obs': {'methaneyield': 43, 'net_energy_requirement': 42},
 'pred': {'methaneyield': 32, 'net_energy_requirement': 46}}

In [107]:
d1 = Entity("AnimalDay", "Cow2", ctx=ctx)
for k,v in asdict(d).items():
    if k == "observedAt":
        #d1[k] = v
        continue
    else:
        d1.prop(k, v)
        d1[k].observedat = start + datetime.timedelta(days=1)
d1.pprint()
#d1.obs()
client.create(d1)

{
  "id": "urn:ngsi-ld:AnimalDay:Cow2",
  "type": "AnimalDay",
  "@context": "http://nginx:8000/context-ngsi.jsonld",
  "obs": {
    "type": "Property",
    "value": {
      "methaneyield": 43,
      "net_energy_requirement": 42
    },
    "observedAt": "2023-01-02T00:00:00Z"
  },
  "pred": {
    "type": "Property",
    "value": {
      "methaneyield": 32,
      "net_energy_requirement": 46
    },
    "observedAt": "2023-01-02T00:00:00Z"
  }
}

True

In [108]:
client.query(type="AnimalDay", ctx=ctx)

[{'@context': 'http://nginx:8000/context-ngsi.jsonld', 'id': 'urn:ngsi-ld:AnimalDay:Cow1', 'type': 'AnimalDay', 'dateObserved': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2023-03-30T08:20:05Z'}}, 'obs': {'type': 'Property', 'value': {'methaneyield': 43, 'net_energy_requirement': 42}}, 'pred': {'type': 'Property', 'value': {'methaneyield': 32, 'net_energy_requirement': 46}}, 'observedAt': {'type': 'Property', 'value': '2022-01-02T00:00:00Z'}},
 {'@context': 'http://nginx:8000/context-ngsi.jsonld', 'id': 'urn:ngsi-ld:AnimalDay:Cow2', 'type': 'AnimalDay', 'obs': {'type': 'Property', 'value': {'methaneyield': 43, 'net_energy_requirement': 42}, 'observedAt': '2023-01-02T00:00:00.000Z'}, 'pred': {'type': 'Property', 'value': {'methaneyield': 32, 'net_energy_requirement': 46}, 'observedAt': '2023-01-02T00:00:00.000Z'}}]

In [64]:
#for t in range(365):
#    d1["observedAt"] = (start + datetime.timedelta(days=-t)).isoformat() + "Z"
#    client.update(d1)

#### Query time series

In [110]:
import requests
import datetime
ctx = "http://nginx:8000/context-ngsi.jsonld"

#Mintaka on mikrok8s
url = 'http://10.0.0.2:8080/temporal/entities/' 
headers = {
    'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': 'AnimalDay',
    'pageSize': 100,
    'lastN': 10,
    'timeproperty': 'observedAt',
    #'timeproperty': 'createdAt',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}',
    'options' : 'temporalValues'
}

%time response = requests.get(url, headers=headers, params=params)
cowdata = response.json()
#cowdata[0]["observedAt"]["values"]
cowdata[0]["obs"]["values"]
response.text

CPU times: user 5.96 ms, sys: 0 ns, total: 5.96 ms
Wall time: 51.1 ms


'[{"id":"urn:ngsi-ld:AnimalDay:Cow2","type":"AnimalDay","obs":{"type":"Property","values":[[{"methaneyield":43,"net_energy_requirement":42},"2023-01-02T00:00:00Z"]]},"pred":{"type":"Property","values":[[{"methaneyield":32,"net_energy_requirement":46},"2023-01-02T00:00:00Z"]]}}]'

In [102]:
cowdata[0]["obs"]["values"]

[[{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.322Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.305Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.286Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.270Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.251Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.235Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.219Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.200Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.180Z'],
 [{'methaneyield': 43, 'net_energy_requirement': 42},
  '2023-03-30T09:05:15.161Z']]